In [ ]:
# !pip install 'sagemaker[local]' sagemaker --upgrade 

In [16]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_serializer


sess = sagemaker.Session()
role = 'arn:aws:iam::514965996716:role/service-role/AmazonSageMaker-ExecutionRole-20181229T135829'# get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [17]:
training_data = 's3://sagemaker-eu-west-1-514965996716/data/plant-disease-dataset/train'
test_data = 's3://sagemaker-eu-west-1-514965996716/data/plant-disease-dataset/valid'

In [20]:
image = '514965996716.dkr.ecr.eu-west-1.amazonaws.com/autogluon-image-classification'

sm_model = Estimator(
    image_name=image,
    role=role, 
    train_instance_count=1, 
    train_instance_type='ml.p2.8xlarge',
    hyperparameters={'epochs': 2, 'ngpus-per-trial': 1},
    sagemaker_session=sess,
    train_use_spot_instances=True,
    train_max_wait=7200,
    train_volume_size=100,
    train_max_run=7200,
)

In [21]:
sm_model.fit({'training': training_data, 'testing': test_data}, wait=False)

In [ ]:
predictor = sm_model.deploy(1, 'ml.m4.xlarge', serializer=json_serializer)

In [ ]:
import pandas as pd
import json
import io

In [ ]:
df = pd.read_csv('s3://datafy-data-lake-public-artifacts/Inc/test.csv')

In [ ]:
data = df.head()[['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]

In [ ]:
record = data.head(1).T.to_dict()[0]

payload = json.dumps(record)

In [ ]:
d = predictor.predict(payload)

In [ ]:
predictor.delete_endpoint()